In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error

import lightgbm as lgb
#import xgboost as xgb
#import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'loss'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 10000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
INPUT = Path("../input/tabular-playground-series-aug-2021")

train = pd.read_csv(INPUT / "train.csv")
test = pd.read_csv(INPUT / "test.csv")
submission = pd.read_csv(INPUT / "sample_submission.csv")

In [5]:
train['lgb_pred'] = np.load("../input/tps-aug-2021-lgb-no-pseudo/lgb_oof.npy")
test['lgb_pred'] = np.load("../input/tps-aug-2021-lgb-no-pseudo/lgb_pred.npy")/10

train['xgb_pred'] = np.load("../input/tps-aug-2021-xgb-no-pseudo-pred/3xgb_oof.npy")+np.load("../input/tps-aug-2021-xgb-no-pseudo-pred/4xgb_oof.npy")
test['xgb_pred'] = (np.load("../input/tps-aug-2021-xgb-no-pseudo-pred/3xgb_pred.npy")+np.load("../input/tps-aug-2021-xgb-no-pseudo-pred/4xgb_pred.npy"))/10

train['ctb_pred'] = np.load("../input/tps-aug-catb-no-pseudo/2ctb_oof.npy")+np.load("../input/tps-aug-catb-no-pseudo/3ctb_oof.npy")
test['ctb_pred'] = (np.load("../input/tps-aug-catb-no-pseudo/2ctb_pred.npy")+np.load("../input/tps-aug-catb-no-pseudo/3ctb_pred.npy"))/10


In [6]:
test.head()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f93,f94,f95,f96,f97,f98,f99,lgb_pred,xgb_pred,ctb_pred
0,250000,0.812665,15,-1.239120,-0.893251,295.5770,15.87120,23.04360,0.942256,29.898000,...,1.69075,1.059300,-3.010570,1.94664,0.529470,1.386950,8.78767,8.120738,7.732295,8.092161
1,250001,0.190344,131,-0.501361,0.801921,64.8866,3.09703,344.80500,0.807194,38.421900,...,1.84351,0.251895,4.440570,1.90309,0.248534,0.863881,11.79390,4.629908,4.877947,4.447047
2,250002,0.919671,19,-0.057382,0.901419,11961.2000,16.39650,273.24000,-0.003300,37.940000,...,1.55100,-0.559157,17.838600,1.83385,0.931796,2.336870,9.05400,8.572120,8.540636,8.321396
3,250003,0.860985,19,-0.549509,0.471799,7501.6000,2.80698,71.08170,0.792136,0.395235,...,1.41390,0.329272,0.802437,2.23251,0.893348,1.359470,4.84833,7.496467,7.472929,7.393689
4,250004,0.313229,89,0.588509,0.167705,2931.2600,4.34986,1.57187,1.118300,7.754630,...,1.58020,-0.191021,26.253000,2.68238,0.361923,1.532800,3.70660,7.207754,6.996287,7.297740


# Preprocessing

In [7]:
scale_features = [col for col in test.columns if 'f' in col]

ss = StandardScaler()
train[scale_features] = ss.fit_transform(train[scale_features])
test[scale_features] = ss.transform(test[scale_features])

In [8]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

In [9]:
useful_features = scale_features + ['lgb_pred', 'xgb_pred','ctb_pred']

# Optuna

In [10]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [11]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=train[useful_features], y=train[target]):
  """
  """
  param_space = {
                    'objective': 'regression',
              'metric':'rmse',
              'n_jobs':-1,
               #'device':'gpu',  # Use GPU acceleration
               # 'gpu_platform_id': 0,
                #    'gpu_device_id': 0,
               'n_estimators':N_ESTIMATORS,
              'learning_rate':trial.suggest_uniform('learning_rate', 1e-2, 1e-1),
               'subsample': trial.suggest_uniform('subsample', 0.67, 1.0),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.356, 0.756),
                'subsample_freq': trial.suggest_int('subsample_freq', 5, 9),
             #'boosting_type': 'gbdt',
               'reg_lambda':trial.suggest_uniform('reg_lambda', 8.55, 32.55),
              'reg_alpha':trial.suggest_uniform('reg_alpha', 30.5, 54.5),
            'min_child_weight':trial.suggest_uniform('min_child_weight', 8, 208),
                 'min_child_samples':trial.suggest_int('min_child_samples', 164, 284),
          'importance_type': 'gain'
                }
            

  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  #lgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  
  pred=np.zeros(y_va.shape[0])
  
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
  
  #X_tr = apply_noise_rn(X_tr)

  
  #start = time.time()
  for inseed in seed_list:
    param_space['random_state'] = inseed

    model = lgb.LGBMRegressor(**param_space)
    model.fit(
        X_tr, 
        y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric='rmse',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
    )


    pred += model.predict(X_va) / len(seed_list)
    
  #elapsed = time.time() - start
  rmse = mean_squared_error(y_va, pred, squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [12]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-28 23:16:51,873] A new study created in memory with name: no-name-6c66f46f-643c-4671-8e53-e016de8183a8


Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[86]	valid_0's rmse: 7.75296
Training until validation scores don't improve for 200 rounds


[I 2021-08-28 23:17:13,684] Trial 0 finished with value: 7.751935784723151 and parameters: {'learning_rate': 0.03876032831466433, 'subsample': 0.9341731704447493, 'colsample_bytree': 0.5541106228231737, 'subsample_freq': 9, 'reg_lambda': 17.551398149476483, 'reg_alpha': 44.64636575856518, 'min_child_weight': 83.67502043238217, 'min_child_samples': 176}. Best is trial 0 with value: 7.751935784723151.


Early stopping, best iteration is:
[96]	valid_0's rmse: 7.75188
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[243]	valid_0's rmse: 7.75144
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[231]	valid_0's rmse: 7.75216


[I 2021-08-28 23:17:46,019] Trial 1 finished with value: 7.751588257735942 and parameters: {'learning_rate': 0.014576588483730888, 'subsample': 0.8313906708649433, 'colsample_bytree': 0.5719171102931042, 'subsample_freq': 8, 'reg_lambda': 29.088531487231364, 'reg_alpha': 49.36011218140722, 'min_child_weight': 137.51081379787257, 'min_child_samples': 208}. Best is trial 1 with value: 7.751588257735942.


...

Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[120]	valid_0's rmse: 7.75327
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[124]	valid_0's rmse: 7.7528


[I 2021-08-28 23:30:12,294] Trial 28 finished with value: 7.75267318471845 and parameters: {'learning_rate': 0.024397805434891028, 'subsample': 0.788090407321264, 'colsample_bytree': 0.6005880126907872, 'subsample_freq': 9, 'reg_lambda': 31.639933753896077, 'reg_alpha': 51.02497908675723, 'min_child_weight': 129.37139229937793, 'min_child_samples': 247}. Best is trial 25 with value: 7.751445395722502.


Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[99]	valid_0's rmse: 7.75222
Training until validation scores don't improve for 200 rounds


[I 2021-08-28 23:30:35,420] Trial 29 finished with value: 7.752096571114211 and parameters: {'learning_rate': 0.04355472003164232, 'subsample': 0.9431199407960068, 'colsample_bytree': 0.5699524094431566, 'subsample_freq': 8, 'reg_lambda': 28.11373286833887, 'reg_alpha': 45.950880697597036, 'min_child_weight': 103.79088212196048, 'min_child_samples': 230}. Best is trial 25 with value: 7.751445395722502.


Early stopping, best iteration is:
[86]	valid_0's rmse: 7.7531
Number of finished trials: 30
Best trial: {'learning_rate': 0.02353337642138889, 'subsample': 0.8927518164857942, 'colsample_bytree': 0.6082521012761244, 'subsample_freq': 8, 'reg_lambda': 30.353719041677138, 'reg_alpha': 51.51192793124132, 'min_child_weight': 191.56782210566885, 'min_child_samples': 237}


In [13]:
study.best_params

{'learning_rate': 0.02353337642138889,
 'subsample': 0.8927518164857942,
 'colsample_bytree': 0.6082521012761244,
 'subsample_freq': 8,
 'reg_lambda': 30.353719041677138,
 'reg_alpha': 51.51192793124132,
 'min_child_weight': 191.56782210566885,
 'min_child_samples': 237}

# Log

7.751556608663328 no noise ver1